<a href="https://colab.research.google.com/github/Zakibrahmi/AgentAI/blob/main/project_PlaningSMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Automated Project: Planning, Estimation, and Allocation**

In this project I will demonstrate an automated approach to project planning, estimation, and resource allocation using the crewAI framework.

To define Agents and Task, I will use YAML files, promoting a structured approach.

The LLM model used is:  **'gpt-4o-mini'**

The multi-agent system is composed of three agents that prompt LLM to achieve their goals:

1. project_planning_agent: to  break down the project into actionable tasks, ensuring no detail is overlooked, and setting precise timelines that align with the project objectives.
2. estimation_agent: provides an accurate time, resource, and effort estimations for each task in the project to ensure it is  delivered efficiently and on budget.
3. resource_allocation_agent: To optimize the allocation of tasks for the  project by balancing team members' skills, availability, and current     workload to maximize efficiency and project success.



In [6]:
!pip install crewai crewai_tools

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
import yaml
from crewai import Agent, Task, Crew

Helper

In [8]:
# This code can be, for example, defined in a file (e.g., helper.py)
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    return openai_api_key


Set the LLM model to be used: 'gpt-4o-mini'

In [15]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
os.environ["OPENAI_API_KEY"] = 'Your_OPenAI_key'

Loading Agents and Tasks descritpion from YAML files.

In [12]:
# Define file paths for YAML configurations
files = {
    'agents': 'agents.yaml',
    'tasks': 'tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

Create Pydantic Models for Structured Output

In [13]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

Now, we can create Crew, Agents and Tasks. Please be correful, *CrewAI* agents often rely on OpenAI LLMs under the hood. This requires setting the OPENAI_API_KEY environment variable before creating Agent instances, even if it's not an explicit argument.  If the API key is missing, a ValidationError will be raised during agent initialization.

In [16]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

To run your SMA, we need Crew's Inputs

In [17]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- Ilyes (Project Manager)
- Adel (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Sabeur (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- Ilyes (Project Manager)
- Adel (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Sabeur (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



Kicking off the crew

In [19]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

 [2025-03-16 02:34:54][DEBUG]: == Working Agent: The Ultimate Project Planner

 [2025-03-16 02:34:54][INFO]: == Starting Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilit

Nice, as you see My Agents powred by LLM model, automaticly create ressource allocation chart for website project:)